In [34]:
import xgboost
import joblib
import numpy as np
import pandas as pd

#### Load model, feature names, testing data and counterfactuals from testing set

In [36]:
#model = joblib.load('files/model_asap7_776')
model = joblib.load('files/model_asap7_776_counter_both_ori')

feature_names = joblib.load('files/feature_names_776')

data_test_ori = pd.read_csv('ASAP7 Test Set.tsv', sep='\t')

feat_test_ori = joblib.load('files/asap7_features_test_776')

feats_counter_test_776 = joblib.load('files/feats_counter_test_correct_776')
np.array(feats_counter_test_776).shape

(100, 31, 1, 776)

In [37]:
feat_test_ori.shape

(314, 776)

In [38]:
# feats_counter_test_776 = np.delete(feats_counter_test, [5,6],axis=3)
# feats_counter_test_776.shape

In [39]:
# joblib.dump(feats_counter_test_776, 'feats_counter_test_776')

#### Predict the scores of the original essays

In [40]:
scores_ori = []
row_ctr = 0

for feat in feat_test_ori[:100]:
    dmat = xgboost.DMatrix(feat.reshape(1,-1), feature_names=feature_names)
    score = model.predict(dmat)
    scores_ori.append(score[0])
    row_ctr = row_ctr + 1
    print("Finished row number : ", row_ctr)

print("row number: ", row_ctr)

Finished row number :  1
Finished row number :  2
Finished row number :  3
Finished row number :  4
Finished row number :  5
Finished row number :  6
Finished row number :  7
Finished row number :  8
Finished row number :  9
Finished row number :  10
Finished row number :  11
Finished row number :  12
Finished row number :  13
Finished row number :  14
Finished row number :  15
Finished row number :  16
Finished row number :  17
Finished row number :  18
Finished row number :  19
Finished row number :  20
Finished row number :  21
Finished row number :  22
Finished row number :  23
Finished row number :  24
Finished row number :  25
Finished row number :  26
Finished row number :  27
Finished row number :  28
Finished row number :  29
Finished row number :  30
Finished row number :  31
Finished row number :  32
Finished row number :  33
Finished row number :  34
Finished row number :  35
Finished row number :  36
Finished row number :  37
Finished row number :  38
Finished row number :

In [41]:
scores_ori[:5]

[13.265924, 20.012081, 11.004908, 16.225878, 8.438831]

In [42]:
joblib.dump(scores_ori, "scores_ori_test_776_both")
# scores_ori = joblib.load("scores_ori_test_776")

['scores_ori_test_776_both']

#### Predict the scores of the counterfactuals

In [43]:
scores_counter = []

row_ctr = 0

for row in feats_counter_test_776:
    row_score = []
    for essay_feat in row:
        essay_feat = xgboost.DMatrix(essay_feat, feature_names=feature_names)
        score = model.predict(essay_feat)
        row_score.append(score[0])
    scores_counter.append(row_score)
    row_ctr = row_ctr + 1
    print("Finished row number : ", row_ctr)

print("row number: ", row_ctr)

Finished row number :  1
Finished row number :  2
Finished row number :  3
Finished row number :  4
Finished row number :  5
Finished row number :  6
Finished row number :  7
Finished row number :  8
Finished row number :  9
Finished row number :  10
Finished row number :  11
Finished row number :  12
Finished row number :  13
Finished row number :  14
Finished row number :  15
Finished row number :  16
Finished row number :  17
Finished row number :  18
Finished row number :  19
Finished row number :  20
Finished row number :  21
Finished row number :  22
Finished row number :  23
Finished row number :  24
Finished row number :  25
Finished row number :  26
Finished row number :  27
Finished row number :  28
Finished row number :  29
Finished row number :  30
Finished row number :  31
Finished row number :  32
Finished row number :  33
Finished row number :  34
Finished row number :  35
Finished row number :  36
Finished row number :  37
Finished row number :  38
Finished row number :

In [23]:
np.array(scores_counter).shape

(100, 31)

In [32]:
joblib.dump(scores_counter, "files/scores_counterfactual_test_correct_776")
#scores_counter = joblib.load('scores_counterfactual_test')

['files/scores_counterfactual_test_correct_776']

#### Compare the scores Original vs Counterfactual

In [46]:
comparison = []

ctr = 1

for idx, row_scores in enumerate(scores_counter):
    ori_score = scores_ori[idx]
    compare_row = []
    for score in row_scores:

        if ctr <= 10:
            print(ctr)
            print("ori: ", ori_score)
            print("counter: ", score)
        
        score_diff = abs(score - ori_score)
        if score >= ori_score:
            compare_row.append((True, score_diff, score))
        else:
            compare_row.append((False, score_diff, score))
        ctr = ctr + 1
        
    comparison.append(compare_row)

1
ori:  13.265924
counter:  22.048935
2
ori:  13.265924
counter:  22.010702
3
ori:  13.265924
counter:  22.038443
4
ori:  13.265924
counter:  22.031889
5
ori:  13.265924
counter:  22.026842
6
ori:  13.265924
counter:  22.048935
7
ori:  13.265924
counter:  22.038443
8
ori:  13.265924
counter:  22.065664
9
ori:  13.265924
counter:  22.048935
10
ori:  13.265924
counter:  22.01635


In [47]:
comparison

[[(True, 8.7830105, 22.048935),
  (True, 8.744778, 22.010702),
  (True, 8.772518, 22.038443),
  (True, 8.7659645, 22.031889),
  (True, 8.760918, 22.026842),
  (True, 8.7830105, 22.048935),
  (True, 8.772518, 22.038443),
  (True, 8.79974, 22.065664),
  (True, 8.7830105, 22.048935),
  (True, 8.750425, 22.01635),
  (True, 8.760918, 22.026842),
  (True, 8.760918, 22.026842),
  (True, 8.772518, 22.038443),
  (True, 8.772518, 22.038443),
  (True, 8.768763, 22.034687),
  (True, 8.772518, 22.038443),
  (True, 8.79974, 22.065664),
  (True, 8.760918, 22.026842),
  (True, 8.770155, 22.03608),
  (True, 8.772518, 22.038443),
  (True, 8.802538, 22.068462),
  (True, 8.750425, 22.01635),
  (True, 8.750425, 22.01635),
  (True, 8.760918, 22.026842),
  (True, 8.775316, 22.04124),
  (True, 8.770155, 22.03608),
  (True, 8.770155, 22.03608),
  (True, 8.777647, 22.043571),
  (True, 8.772953, 22.038877),
  (True, 8.748062, 22.013987),
  (True, 8.770155, 22.03608)],
 [(True, 3.8897057, 23.901787),
  (True, 3.8

#### Count the number of False and mean of score difference for the 'False' predictions

In [48]:
# Initialize counters and lists to hold False values' second column values
false_count = 0
false_values = []
new_label = []
true_count = 0

# Iterate through each sublist
for idx, sublist in enumerate(comparison):
    ori = scores_ori[idx]
    row_label = []
    for value in sublist:

        formula = ori + value[1]
        
        if value[0] == False:  # If the first element is False
            false_count += 1
            false_values.append(value[1])  # Collect the second element (the difference)
            row_label.append(formula)
        else:
            true_count += 1
            row_label.append(value[2])
    new_label.append(row_label)
    
# Calculate the mean of the differences for False values only
if false_count > 0:
    mean_difference = np.mean(false_values)
else:
    mean_difference = 0  # Handle case where there are no False values

print(f"Count of False values: {false_count}")
print(f"Count of True values: {true_count}")
print(f"Mean of the differences for False values: {mean_difference}")


Count of False values: 1347
Count of True values: 1753
Mean of the differences for False values: 4.627378463745117


In [49]:
np.array(comparison).shape

(100, 31, 3)

In [50]:
acc = (true_count / 3100) * 100
acc

56.54838709677419

In [30]:
np.array(new_label).shape

(100, 31)

In [40]:
#joblib.dump(new_label, 'new_label_test_776')